In [4]:
"""
This script is designed to prevent the usage of Activity-Browser while realize the same function as Test_bwLCA.ipynb.
Author: Ao Chen. Place: UZH Zentrum. 
"""
import os
import pandas as pd
from brightway2 import *
from datetime import date
import bw2data

In [5]:
def project_setup(Project_name, Dataset_path, Database_name):
    projects.set_current(Project_name) 
    if len(Database("biosphere3")) == 0: # if no biosphere3 database, set it up.
        bw2setup()
    if len(Database(Database_name)) == 0:
        ei38cutoff = SingleOutputEcospold2Importer(
            Dataset_path,
            Database_name
        ) # import ecoinvent database
        ei38cutoff.apply_strategies()
        ei38cutoff.statistics()
        ei38cutoff.write_database()

In [6]:
Project_name = "Ecoinvent" # if not exist, automatically create one
Dataset_path = "/Users/ocean/Library/CloudStorage/OneDrive-forpersonal/RA-SustainableConstruction/bwLCA/ecoinvent 3.8_cutoff_ecoSpold02/datasets"
# TODO: substitute this with your ecoinvent datasets path (absolute path is more reliable)
Database_name = "cutoff38"
project_setup(Project_name, Dataset_path, Database_name)

In [7]:
db = Database(Database_name)
Process_list_all = db.search("*", limit=10000000) # read all processes in the database
# TODO: this process is time consuming, run once is enough!

In [8]:
class LCA_Calculation():
    def __init__(self, Process_list_all, Impact_method, Result_path, Excel_path='', Process_list_target=None):
        self.Process_list_all = Process_list_all
        self.Impact_method = Impact_method
        self.Result_path = Result_path
        self.Excel_path = Excel_path
        self.Process_list_target = Process_list_target

        if not os.path.exists(self.Result_path):
            os.makedirs(self.Result_path)
    
    def cal_from_excel(self):
        material_df = pd.read_excel(self.Excel_path, header=None)
        material_df[['Activity', 'Product', 'Cutoff']] = material_df[0].str.split('|', 2, expand=True)
        material_df['Activity'] = material_df['Activity'].str.strip()

        results = {}
        activities = {}
        # for i in range(len(material_df)):
        for i in range(1): # for test
            search_process_list = [p for p in self.Process_list_all if material_df['Activity'][i] in str(p)] # search for processes with specific name
            for j in range(len(search_process_list)):
                functional_unit = {search_process_list[j]: 1}
                lca = LCA(functional_unit, self.Impact_method)
                lca.lci()
                lca.lcia()
                results[dict(search_process_list[j])['location']] = lca.score
                activities[dict(search_process_list[j])['location']] = dict(search_process_list[j])['name']
            
            results_Series = pd.Series(results, name='kg CO2-Eq')
            activities_Series = pd.Series(activities, name='activity')
            output_df = pd.concat([results_Series, activities_Series], axis=1)
            output_df.to_csv(self.Result_path+'/Summary_'+material_df['Activity'][i]+'.csv')
            print('Results stored in' + str(os.path.abspath(self.Result_path+'/Summary_'+material_df['Activity'][i]+'.csv')))
            
    def cal_from_list(self):
        results = {}
        activities = {}

        for i in range(len(self.Process_list_target['include'])):
            search_process_list = [p for p in self.Process_list_all if self.Process_list_target['include'][i] in str(p) and not self.Process_list_target['not_include'][i] in str(p)]
            for j in range(len(search_process_list)):
                functional_unit = {search_process_list[j]: 1}
                lca = LCA(functional_unit, self.Impact_method)
                lca.lci()
                lca.lcia()
                results[dict(search_process_list[j])['location']] = lca.score
                activities[dict(search_process_list[j])['location']] = dict(search_process_list[j])['name']

            results_Series = pd.Series(results, name='kg CO2-Eq')
            activities_Series = pd.Series(activities, name='activity')
            output_df = pd.concat([results_Series, activities_Series], axis=1)
            output_df.to_csv(self.Result_path+'/Summary_'+self.Process_list_target['include'][i]+'.csv')
            print('Results stored in' + str(os.path.abspath(self.Result_path+'/Summary_'+self.Process_list_target['include'][i]+'.csv')))

In [11]:
Impact_method = [m for m in methods if 'IPCC 2013' in str(m) and 'climate change' in str(m) and 'GWP 100a' in str(m) and not 'no LT' in str(m)][0]
Today = date.today().strftime("%b_%m_%Y")
Result_path = Today+'_calc_results_GWP100a_test'

Excel_path = 'List_of_Materials_ecoinvent.xlsx' # optional
Process_list_target = {
    'include': ['market for electricity, medium voltage'], # key words you want to include
    'not_include': ['municipal waste'] # key words you do not want to include
} # optional

instance1 = LCA_Calculation(Process_list_all, Impact_method, Result_path, Excel_path=Excel_path, Process_list_target=Process_list_target)
instance1.cal_from_excel() # input Excel
instance1.cal_from_list() # input list (in fact a dict)

# TODO: change names above as you need

Results stored in/Users/ocean/Library/CloudStorage/OneDrive-forpersonal/RA-SustainableConstruction/bwLCA/Feb_02_2022_calc_results_GWP100a_test/Summary_concrete block production.csv
Results stored in/Users/ocean/Library/CloudStorage/OneDrive-forpersonal/RA-SustainableConstruction/bwLCA/Feb_02_2022_calc_results_GWP100a_test/Summary_market for electricity, medium voltage.csv


In [10]:
# projects.delete_project(name='Test_Ecoinvent', delete_dir=True) # Delete everything in the project. TODO: only uncomment this line when necessary!!!

'Ecoinvent'